# Longitude Conventions and Data Subsetting
This notebook demonstrates how to handle longitude conventions (0–360 and -180–180 degrees) and how to subset spatial and temporal data using climatrix.

In [2]:
import climatrix as cm
import xarray as xr
import numpy as np
import pandas as pd
from climatrix.dataset.axis import AxisType
from datetime import datetime

## Creation of simple Dataset

In [3]:
xarray_da = xr.DataArray(
    data=np.random.rand(1, 5, 5),
    dims=("time", "latitude", "longitude"),
    coords={
        "time": np.array(
            [
                datetime(2000, 1, 1),
            ],
            dtype="datetime64",
        ),
        "latitude": (
            ("latitude",),
            np.array([-90, -45, 0, 45, 90]),
        ),
        "longitude": (
            ("longitude",),
            np.array([-180, -90, 0, 90, 180]),
        ),
    },
)
cm_dset = xarray_da.to_dataset(name="temperature").cm
print("\n-180-180 convention dataset:")
print(cm_dset.da.longitude.values)


-180-180 convention dataset:
[-180  -90    0   90  180]


## Conversion of Longitude Conventions

In [4]:
cm_positive_conv = cm_dset.to_positive_longitude()
print("\nConverted to positive longitude convention:")
print(cm_positive_conv.da.longitude.values)


cm_signed_conv = cm_positive_conv.to_signed_longitude()
print("Converted to signed longitude convention::")
print(cm_signed_conv.da.longitude.values)


Converted to positive longitude convention:
[ 90 180 180 270   0]
Converted to signed longitude convention::
[-180  -90    0   90 -180]


## Spatial Subsetting (`subset`)

In [5]:
cm_data_for_subset = cm_dset
subset_result = cm_data_for_subset.subset(
    north=60, south=10, west=-100, east=-50
)

print(subset_result.da)

<xarray.DataArray 'temperature' ()> Size: 8B
array(0.90811081)
Coordinates:
    latitude   int64 8B 45
    longitude  int64 8B -90


## Temporal Subsetting (`time` and `itime`)

In [11]:
cm_temporal = cm_dset

# TODO it will work only for dynamic datasets move it to dynamic examples
selected_time_dt = cm_temporal.time(datetime(2000, 1, 1))
print(selected_time_dt.da.time.values)

selected_time_slice = cm_temporal.time(slice("2000-01-01", "2000-01-03"))
print(selected_time_slice.da.time.values)

selected_time_list = cm_temporal.time([pd.Timestamp("2000-01-01"), pd.Timestamp("2000-01-04")])
print(selected_time_list.da.time.values)

selected_itime_single = cm_temporal.itime(0)
print(selected_itime_single.da.time.values)

selected_itime_slice = cm_temporal.itime(slice(1, 3))
print(selected_itime_slice.da.time.values)

KeyError: "no index found for coordinate 'time'"

## Generic Selections `sel` and `isel`
These methods allow the use of AxisType or axis names defined in climatrix for data selection.

In [25]:
sel_result = cm_dset.sel({
    AxisType.LATITUDE: slice(-45, 45),
    'longitude': 0,
})
print(f"sel results:\n{sel_result.da}")


isel_result = cm_temporal.isel({
    'latitude': slice(0,2)
})
print(f"\nisel results:\n{isel_result.da}")

sel results:
<xarray.DataArray 'temperature' (latitude: 3)> Size: 24B
array([0.78002562, 0.07385644, 0.60608568])
Coordinates:
  * latitude   (latitude) int64 24B -45 0 45
    longitude  int64 8B 0

isel results:
<xarray.DataArray 'temperature' (latitude: 2, longitude: 5)> Size: 80B
array([[0.46486655, 0.05835932, 0.58324094, 0.80371   , 0.15166639],
       [0.91271243, 0.08020387, 0.78002562, 0.24922762, 0.71073751]])
Coordinates:
  * latitude   (latitude) int64 16B -90 -45
  * longitude  (longitude) int64 40B -180 -90 0 90 180
